## Imports

In [ ]:
import pandas as pd
from datetime import datetime as dt
from hbt import geometry as gm
from hbt import utils_hbt as uh
from pathlib import Path
from plotly import graph_objects as go
from plotly.subplots import make_subplots

## Functions

In [ ]:
def compute_stats_pw(data_pw, info_pw, night_start='00:00',
    night_end='06:00'):
    cols = ['Type', 'Mean DH+ day [cm]', 'Mean DH+ night [cm]',
        'Mean DH- day [cm]', 'Mean DH- night [cm]', 'Mean inflow day [l/s]',
        'Mean inflow night [l/s]', 'Mean outflow day [l/s]',
        'Mean outflow night [l/s]']

    pw_stats = pd.DataFrame(index=data_pw.columns, columns=cols)

    pw_night = data_pw.between_time(night_start, night_end, inclusive='left')
    pw_day = data_pw.between_time(night_end, night_start, inclusive='left')
    pw_night = pw_night.diff()
    pw_day = pw_day.diff()

    pw_stats.iloc[:,1] = pw_day[pw_day>0].mean()
    pw_stats.iloc[:,2] = pw_night[pw_night>0].mean()
    pw_stats.iloc[:,3] = pw_day[pw_day<0].mean()
    pw_stats.iloc[:,4] = pw_night[pw_night<0].mean()

    names = data_pw.columns
    for pw, info in info_pw.items():
        pw_stats.loc[pw, 'Type'] = info['typ']
        if info['area']:
            idx = pw_stats.index.get_loc(pw)
            pw_stats.iloc[idx, 5:] = (pw_stats.iloc[idx, 1:5] / 10
                * info['area'] * 100) / 60

    return pw_stats

## Settings

In [ ]:
path_pwdata = Path(r"Q:\Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\5 Fremdwasser\Pumpwerke"
    r"\220722_10.56Uhr_Niveau_PW_1-jahr.csv")

path_raindata = Path(r"Q:\Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\5 Fremdwasser\Pumpwerke\data_waedenswil.csv")

save_dir = Path(r"Q:\Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\5 Fremdwasser\Pumpwerke")

info_pw = { # nur SW Pumpwerke
    'PW Hüttner See': {
        'area': gm.circ_area_d(2.5),
        'typ': 'SW',
    },
    'PW Seegarten': {
        'area': 2.1 * 1.4,
        'typ': 'RW',
    },
    'PW Mülibach': {
        'area': 3.35 * 2,
        'typ': 'SW',
    },
    'PW Mülibach Wand': {
      'area': 1.9 * 1.0,
      'typ': 'RW', 
    },
    'PW Klosterweg': {
        'area': 2.0 * 3.0,
        'typ': 'SW',
    },
    'PW Stationsstrasse': {
        'area': gm.circ_area_d(1.25),
        'typ': 'SW',
    },
    'PW Schmittenbrunnen': {
        'area': 0.8 * 1.5,
        'typ': 'SW',
    },
    'PW SBB Wand': {
        'area': 1.5 * 2.5,
        'typ': 'RW',
    },
    'PW Streuli': {
        'area': 1.0 * 1.8,
        'typ': 'SW',
    },
    'PW Mülenen': {
        'area': 1.65 * 2.2, # variabel, ca. 1.65 * 2.20
        'typ': 'SW',
    },
    'PW SBB Mülenen': {
        'area': 2.6 * 1.8,
        'typ': 'RW',
    },
    'PW Hirzen': {
        'area': gm.circ_area_d(1.25),
        'typ': 'SW',
    },
    'PW Seeli': {
        'area': gm.circ_area_d(1.25),
        'typ': 'SW',
    },
    'PW Trüliplatz': {
        'area': gm.circ_area_d(1.0),
        'typ': 'SW',
    },
    'PW Horn': {
        'area': gm.circ_area_d(1.25),
        'typ': 'SW',
    },
    'PW Forsthaus': {
        'area': None,
        'typ': 'SW',
    },
}

## Read data

In [ ]:
date_format = '%d.%m.%Y %H:%M'
date_parser = lambda x: pd.to_datetime(x, dayfirst=True, format=date_format)
data_pw = pd.read_csv(path_pwdata, sep=';', header=0, index_col=0,
    parse_dates=True, date_parser=date_parser, dayfirst=True)

# Rename columns
f_rename = lambda x: x.split(';')[0]
data_pw.rename(columns=f_rename, inplace=True)

# Convert m -> cm
data_pw = data_pw * 100

In [ ]:
ts_rain = uh.read_raindata_agrometeo(path_raindata, resolution='10min')
ts_rain['cumsum'] = ts_rain.cumsum()

## Filter data

In [ ]:
bool_filter = True
if bool_filter:
    date_from = dt(2021, 7, 1)
    date_to = dt(2021, 7, 31)
    dt_filter = lambda df: df[(df.index >= date_from) & (df.index <= date_to)]

    data_pw = dt_filter(data_pw)
    ts_rain = dt_filter(ts_rain)

## Analysis

In [ ]:
pw_stats = compute_stats_pw(data_pw, info_pw)

save_path = save_dir / 'Mean_PW_inflows-outflows_RW.csv'
pw_stats.iloc[:,1:] = pw_stats.iloc[:,1:].astype(float).round(1)
pw_stats.to_csv(save_path, sep=';', encoding='latin1')

## Plots

### Plot data

In [ ]:
plot = True
if plot:
    names = data_pw.columns
    for name in names:
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        fig.add_trace(go.Scatter(x=data_pw.index, y=data_pw[name], name=name),
            secondary_y=False)
        fig.add_trace(go.Scatter(x=ts_rain.index, y=ts_rain['cumsum'],
            name='Regen kumulativ'), secondary_y=True)
        fig.update_xaxes(title='Zeit')
        fig.update_yaxes(title='Niveau [cm]', secondary_y=False)
        fig.update_yaxes(title='Regenmenge [mm]', secondary_y=True)
        fig.update_layout(title=f'{name}')

        prefix = info_pw[name]['typ']
        name = f'{prefix}-{name}'
        save_path = save_dir / f'Plots_RW/{name}.html'
        fig.write_html(save_path)